In [1]:
import frank_lab

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(rc={'figure.figsize': (12, 6),'lines.linewidth': 2, 'font.size': 18, 'axes.labelsize': 16, 'legend.fontsize': 12, 'ytick.labelsize': 12, 'xtick.labelsize': 12 })
sns.set_style('white')


In [3]:
import numpy as np

In [4]:
fileroot = '/Users/ckemere/DataAnalysis/Fra';

In [5]:
import os.path
import scipy.io
import pandas as pd
import numpy as np
import re
import glob

def load_spike(fileroot, day=0) :
    anim_prefix = frank_lab.get_animal_prefix(fileroot);

    if (day == 0) :
        all_spikes = pd.DataFrame()
        file_list = []
        day_list = []
        for file in os.listdir(fileroot):
            m = re.match("{}task([0-9]+)\.mat".format(anim_prefix), file, re.IGNORECASE)
            if m:
                dk = int(m.group(1))
                all_spikes = all_spikes.append(load_spike(fileroot, dk), ignore_index=True, verify_integrity=True)

        #task = task.sort_values(by=['Day','Epoch'])

        all_spikes = all_spikes.sort_values(by=['Day','Epoch','Tetrode','Cell'])
        cols = all_spikes.columns.tolist()
        newcols = ['Day','Epoch','Tetrode','Cell']
        newcols = newcols + list(set(cols) - set(newcols))

        return all_spikes[newcols]

    else :
        filename = "{}spikes{:02d}.mat".format(anim_prefix, day)

        print("Loading " + filename)
        mat = scipy.io.loadmat(os.path.join(fileroot,filename), 
                               struct_as_record=False, squeeze_me=True)

        spikedata = []
        # Spike files contain data for all epochs, tetrodes, and cells in a day
        # Some epochs have fields: ['data', 'descript', 'fields', 'depth', 'spikewidth', 'timerange']
        # but some epochs are missing 'spikewidth'
        data = mat['spikes']
        data = data[day-1]
        for epidx, da in enumerate(data):
            for tetidx, te in enumerate(da):
                if isinstance(te, np.ndarray) :
                    for cellidx, cell in enumerate(te):
                        spikedata.append({})
                        spikedata[-1]['Day'] = day
                        spikedata[-1]['Epoch'] = epidx
                        neuron_idx = (day-1, epidx, tetidx, cellidx)
                        spikedata[-1]['Tetrode'] = tetidx
                        spikedata[-1]['Cell'] = cellidx
                        if (isinstance(cell, np.ndarray)) :
                            continue # No data for this tetrode/cell combo
                        spikedata[-1].update({f: getattr(cell,f) for f in cell._fieldnames})
                        if cell.data.size == 0 :
                            spikedata[-1]['spiketimes'] = cell.data # this is an empty array
                        else :
                            if (cell.data.ndim == 1) :
                                spikedata[-1]['spiketimes'] = cell.data[0]
                            else :
                                spikedata[-1]['spiketimes'] = cell.data[:,0]

                else: # Single cell on tetrode
                    spikedata.append({})
                    spikedata[-1]['Day'] = day
                    spikedata[-1]['Epoch'] = epidx
                    neuron_idx = (day-1, epidx, tetidx, 0)
                    spikedata[-1]['Tetrode'] = tetidx
                    spikedata[-1]['Cell'] = 0
                    spikedata[-1].update({f: getattr(te,f) for f in te._fieldnames})
                    if te.data.size == 0 :
                        spikedata[-1]['spiketimes'] = te.data # this is an empty array
                    else : 
                        if (te.data.ndim == 1) :
                            spikedata[-1]['spiketimes'] = te.data[0]
                        else :
                            spikedata[-1]['spiketimes'] = te.data[:,0]

        return spikedata

In [6]:
spikes = frank_lab.load_data(fileroot, datatype='spikes', day=0)

Loading fraspikes02.mat
Loading fraspikes03.mat
Loading fraspikes04.mat
Loading fraspikes05.mat
Loading fraspikes06.mat
Loading fraspikes07.mat
Loading fraspikes08.mat
Loading fraspikes09.mat
Loading fraspikes10.mat
Loading fraspikes11.mat
Loading fraspikes12.mat


In [7]:
spikes

,Day,Epoch,Tetrode,Cell,spikewidth,descript,data,spiketimes,timerange,depth,fields
0,2,0,0,0,6.659571,spike data,"[[1742.4857, 122.85, 83.25, 0.636508215788, 0....","[1742.4857, 1742.4978, 1742.5475, 1770.9462, 1...","[15530000, 23820000]",123.0,time x y dir thetaphase amplitude(highest vari...
1,2,0,0,1,8.108285,spike data,"[[1566.5365, 134.1, 114.3, 2.62244653934, 0.00...","[1566.5365, 1566.8038, 1566.8083, 1566.8121, 1...","[15530000, 23820000]",123.0,time x y dir thetaphase amplitude(highest vari...
2,2,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,0,1,0,8.239879,spike data,"[[1889.0401, 135.45, 107.55, 1.46771471827, 5....","[1889.0401, 1897.333, 1897.3386, 1919.9323, 19...","[15530000, 23820000]",152.0,time x y dir thetaphase amplitude(highest vari...
4,2,0,1,1,8.406807,spike data,"[[1554.3284, 133.65, 114.3, 2.6066375798, 3.51...","[1554.3284, 1554.3406, 1578.078, 1666.0552, 16...","[15530000, 23820000]",152.0,time x y dir thetaphase amplitude(highest vari...
5,2,0,1,2,9.321114,spike data,"[[1692.6661, 125.1, 113.4, -2.89661399046, 5.3...","[1692.6661, 1709.5495, 1751.9228, 1754.8869, 1...","[15530000, 23820000]",152.0,time x y dir thetaphase amplitude(highest vari...
6,2,0,1,3,8.880677,spike data,"[[1751.6042, 122.4, 82.8, 0.628796286415, 2.15...","[1751.6042, 1751.6094, 1751.6134, 1767.4336, 1...","[15530000, 23820000]",152.0,time x y dir thetaphase amplitude(highest vari...
7,2,0,7,0,7.776129,spike data,"[[1698.6305, 132.75, 96.75, -0.222081901905, 6...","[1698.6305, 1698.8707, 1700.518, 1700.9424, 17...","[15530000, 23820000]",159.0,time x y dir thetaphase amplitude(highest vari...
8,2,0,7,1,5.337470,spike data,"[[1553.558, 133.2, 114.75, 2.59593759906, 3.41...","[1553.558, 1553.6082, 1553.692, 1553.7349, 155...","[15530000, 23820000]",159.0,time x y dir thetaphase amplitude(highest vari...
9,2,0,20,0,6.922388,spike data,"[[1678.3184, 127.8, 92.25, -3.04811587243, 0.6...","[1678.3184, 1858.2444, 1858.2521, 1858.3428, 1...","[15530000, 23820000]",117.0,time x y dir thetaphase amplitude(highest vari...


In [8]:
ss = spikes['spiketimes'].iloc[0]

In [9]:
[cellinfo,rawcellinfo] = frank_lab.load_data(fileroot, datatype='cellinfo')



Loading fracellinfo00.mat


In [10]:
cellinfo


,Cell,meanrate,subarea,numspikes,Day,hemisphere,Tetrode,nearCA2,spikewidth,area,Epoch,type
"(1, 0, 0, 0)",0,0.037394,CA2,31.0,1,right,0,1.0,6.65957,CA2,0,principal
"(1, 0, 0, 1)",1,0.481303,CA2,399.0,1,right,0,1.0,8.10829,CA2,0,principal
"(1, 0, 1, 0)",0,0.044632,CA2-CA3a,37.0,1,right,1,1.0,8.23988,CA2,0,principal
"(1, 0, 1, 1)",1,0.219542,CA2-CA3a,182.0,1,right,1,1.0,8.40681,CA2,0,principal
"(1, 0, 1, 2)",2,0.059107,CA2-CA3a,49.0,1,right,1,1.0,9.32111,CA2,0,principal
"(1, 0, 1, 3)",3,0.060314,CA2-CA3a,50.0,1,right,1,1.0,8.88068,CA2,0,principal
"(1, 0, 20, 0)",0,0.037394,CA2-CA3a,31.0,1,left,20,1.0,6.92239,CA2,0,principal
"(1, 0, 26, 0)",0,0.324487,CA2-CA3a,269.0,1,right,26,1.0,7.46446,CA2,0,principal
"(1, 0, 26, 1)",1,1.254524,CA2-CA3a,1040.0,1,right,26,1.0,8.24167,CA2,0,principal
"(1, 0, 26, 2)",2,0.032569,CA2-CA3a,27.0,1,right,26,1.0,8.24666,CA2,0,principal
